In [1]:
'''
HMT demo script, broadly used to prep RDMF indexed data for analysts to use
'''
from google.cloud import bigquery
import pandas as pd

In [12]:
# EPC georef query
# will also need address_entry_id to pull in address index info

query = ("""
SELECT

UPPER(lmk_key) AS lmk_key,
guid,
UPPER(current_energy_rating) AS current_energy_rating,
UPPER(potential_energy_rating) AS potential_energy_rating,
UPPER(REGEXP_REPLACE(construction_age_band, r'England and Wales: ', ''))
AS construction_age_band

FROM `ingest_epc_std_georef.epc_2022_03`

""")

query_job = client.query(
    query,
    location="europe-west2",
)  # API request - starts the query


In [47]:
# joining georef and std lrpp tables
query = ("""
SELECT 
 *
FROM `ingest_ida_land_registry_price_paid.prices_paid_georef` t1

JOIN `ingest_ida_land_registry_price_paid.prices_paid_std` t2
ON t1.id = t2.guid

""")
table_ref = 'ons-ids-analysis-prod.demo_wip_notebook.lrpp_linked'

job_config = bigquery.QueryJobConfig(
    destination= table_ref
)

query_job = client.query(
    query,
    location="europe-west2",
    job_config=job_config
)  # API request - starts the query


query_job.result()

In [51]:
# joining OSPOS and (newly joined) LRPP tables
query = ("""
SELECT 

*

EXCEPT (geography_entry_id_postcode, uprn_hashed, postcode_hashed, guid, property_type, address_entry_id_uprn)

FROM `ons-ids-analysis-prod.demo_wip_notebook.lrpp_linked` t1

JOIN `ingest_ida_ordnance_survey_private_outside_space.april_gb_private_outdoor_space_std` t2

ON t1.address_entry_id_uprn = t2.address_entry_id_uprn;
    
""")
table_ref = 'ons-ids-analysis-prod.demo_wip_notebook.ospos_lrpp_linked'

job_config = bigquery.QueryJobConfig(
    destination= table_ref
)

query_job = client.query(
    query,
    location="europe-west2",
    job_config=job_config
)  # API request - starts the query

query_job.result()

In [61]:
# querying ospos lrpp linked data
query = ("""
SELECT 

ur2fold, ur3fold, ur6fold, ur8fold

FROM `ons-ids-analysis-prod.demo_wip_notebook.ospos_lrpp_linked`
    
""")
table_ref = 'ons-ids-analysis-prod.demo_wip_notebook.ospos_lrpp_linked'

query_job = client.query(
    query,
    location="europe-west2",
)  # API request - starts the query

df = query_job.to_dataframe()

In [62]:
df[df.ur2fold != '']

,ur2fold,ur3fold,ur6fold,ur8fold
155395,2,3,6,8
530201,2,3,6,8
624295,2,3,6,8
624389,2,2,5,6
784188,1,1,2,2
...,...,...,...,...
22318104,2,3,6,8
22434681,1,1,1,1
22445613,2,3,6,8
22908829,2,2,5,6


In [59]:
df[df.lsoa_code_hashed == '']

,lsoa_code_hashed,oa_code_hashed,lad_code_hashed,msoa_code_hashed,country_code_hashed,region_code_hashed
0,,,,,,
1,,,,,,
2,,,,,,
3,,,,,,
4,,,,,,
...,...,...,...,...,...,...
25836155,,,,,,
25836156,,,,,,
25836157,,,,,,
25836158,,,,,,


In [ ]:
# for demo purposes, match rate would be useful

match_rate = len(df)/len(source_df)*100

print("match rate:", match_rate)

In [ ]:
# exploring LRPP join
query = ("""
SELECT 
COUNT(*) as numRecords

FROM `ingest_ida_land_registry_price_paid.prices_paid_georef` t1
JOIN `ingest_ida_land_registry_price_paid.prices_paid_std` t2

ON t1.id = t2.guid;
""")

query_job = client.query(
    query,
    location="europe-west2",
)  # API request - starts the query

df  = query_job.to_dataframe()

In [30]:
# all 26889532 records join
df

,numRecords
0,26889532


In [16]:
# LRPP std query
query = ("""
SELECT * 

FROM `ingest_ida_land_registry_price_paid.prices_paid_std`

WHERE 1 = 0
""")

query_job = client.query(
    query,
    location="europe-west2",
)  # API request - starts the query

df  = query_job.to_dataframe()


In [17]:
df.columns.tolist()

['geography_entry_id_postcode',
 'postcode_hashed',
 'guid',
 'transaction_uid',
 'price',
 'date_of_transfer',
 'property_type',
 'old_new',
 'duration',
 'locality',
 'ppd_category_type',
 'derived_year']

In [36]:
# OSPOS query
query = ("""
SELECT * 

FROM `ingest_ida_ordnance_survey_private_outside_space.april_gb_private_outdoor_space_std`

WHERE 1 = 0
""")

query_job = client.query(
    query,
    location="europe-west2",
)  # API request - starts the query

df  = query_job.to_dataframe()


In [37]:
df

,address_entry_id_uprn,geography_entry_id_lsoa_code,geography_entry_id_postcode,geography_entry_id_oa_code,geography_entry_id_lad_code,geography_entry_id_msoa_code,geography_entry_id_country_code,geography_entry_id_region_code,uprn_hashed,lsoa_code_hashed,...,ruc_code,ruc,ur2fold,ur3fold,ur6fold,ur8fold,property_type,building_flat_count,private_outdoor_space,private_outdoor_space_area
